In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [3]:
BATCH_SIZE = 512
EPOCHS = 20
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
train_loader = torch.utils.data.DataLoader(datasets.MNIST('data',train = True, download = True, transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,),(0.3081,))])),batch_size = BATCH_SIZE, shuffle = True)

In [5]:
test_loader = torch.utils.data.DataLoader(datasets.MNIST('data', train = False, transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,),(0.3081,))])), batch_size = BATCH_SIZE, shuffle = True)

In [12]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,10,5)
        self.conv2 = nn.Conv2d(10,20,3)
        self.fc1 = nn.Linear(20*10*10,500)
        self.fc2 = nn.Linear(500,10)
        
        
    def forward(self,x):
        in_size = x.size(0)
        out = self.conv1(x)
        out = F.relu(out)
        out = F.max_pool2d(out,2,2)
        out = self.conv2(out)
        out = F.relu(out)
        out = out.view(in_size,-1)
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        oout = F.log_softmax(out,dim = 1)
        return out

In [13]:
model = ConvNet().to(DEVICE)
optimizer = optim.Adam(model.parameters())

In [14]:
def train(model,device,train_loader,optimizer,epoch):
    model.train()
    for batch_idx, (data,target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if (batch_idx + 1)%30 ==0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss:{:.6f}'.format(epoch,batch_idx * len(data), len(train_loader.dataset),100.* batch_idx / len(train_loader), loss.item()))

In [17]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction = 'sum').item()
            pred = output.max(1, keepdim = True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            test_loss /= len(test_loader.dataset)
            print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{}({:.0f}%)\n'.format(test_loss,correct, len(test_loader.dataset), 100.* correct / len(test_loader.dataset)))

In [18]:
for epoch in range(1, EPOCHS + 1):
    train(model, DEVICE, train_loader, optimizer, epoch)
    test(model, DEVICE, test_loader)

Train Epoch: 1 [14848/60000 (25%)]	Loss:-4076422.250000
Train Epoch: 1 [30208/60000 (50%)]	Loss:-10370094.000000
Train Epoch: 1 [45568/60000 (75%)]	Loss:-23060544.000000

Test set: Average loss: -2253178.4704, Accuracy: 52/10000(1%)


Test set: Average loss: -2282615.0266, Accuracy: 103/10000(1%)


Test set: Average loss: -2209540.6199, Accuracy: 156/10000(2%)


Test set: Average loss: -2227501.2357, Accuracy: 216/10000(2%)


Test set: Average loss: -2235067.5245, Accuracy: 263/10000(3%)


Test set: Average loss: -2226641.1708, Accuracy: 310/10000(3%)


Test set: Average loss: -2248899.4257, Accuracy: 349/10000(3%)


Test set: Average loss: -2237619.4243, Accuracy: 402/10000(4%)


Test set: Average loss: -2309699.4995, Accuracy: 454/10000(5%)


Test set: Average loss: -2265694.9827, Accuracy: 498/10000(5%)


Test set: Average loss: -2266731.1711, Accuracy: 536/10000(5%)


Test set: Average loss: -2206322.0331, Accuracy: 607/10000(6%)


Test set: Average loss: -2255963.4034, Accuracy: 6